## CLEFIA CPA Attack

```
Conduct Correlation Power Analysis attacks at the first, second and last rounds of CLEFIA-128 encryption. 

Authors: Arjun Menon V (ee18b104), Akilesh Kannan (ee18b122)
```

Assignment 2, Secure Processor Microarchitecture

August 2022

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from tqdm.notebook import tnrange
from scipy.stats import linregress
import seaborn as sns
import time
import rich as r
import pandas as pd
from IPython.display import clear_output # type: ignore
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.palettes import brewer

In [3]:
# Load input and power trace
textInp = np.load("../trace_002/textin_array.npy", "r")
powerTrace = np.load("../trace_002/trace_array.npy", "r")
cipherText = np.load("../trace_002/textout_array.npy", "r")
#
s0Arr = np.load("../data/s0.npy", "r")
s1Arr = np.load("../data/s1.npy", "r")
#
tf00Arr = np.load("../data/tf00.npy", "r")
tf01Arr = np.load("../data/tf01.npy", "r")
tf02Arr = np.load("../data/tf02.npy", "r")
tf03Arr = np.load("../data/tf03.npy", "r")
#
tf10Arr = np.load("../data/tf10.npy", "r")
tf11Arr = np.load("../data/tf11.npy", "r")
tf12Arr = np.load("../data/tf12.npy", "r")
tf13Arr = np.load("../data/tf13.npy", "r")

print(textInp.shape)
print(powerTrace.shape)

(12000, 16)
(12000, 18000)


In [4]:
# Split the 8 T-Tables into four, each holding a byte of the result. In all there are 32 T-Tables

tf000Arr = np.array([val & 0xFF for val in tf00Arr])
tf001Arr = np.array([(val & 0xFF00)>>8 for val in tf00Arr])
tf002Arr = np.array([(val & 0xFF0000)>>16 for val in tf00Arr])
tf003Arr = np.array([(val & 0xFF000000)>>24 for val in tf00Arr])

tf010Arr = np.array([val & 0xFF for val in tf01Arr])
tf011Arr = np.array([(val & 0xFF00)>>8 for val in tf01Arr])
tf012Arr = np.array([(val & 0xFF0000)>>16 for val in tf01Arr])
tf013Arr = np.array([(val & 0xFF000000)>>24 for val in tf01Arr])

tf020Arr = np.array([val & 0xFF for val in tf02Arr])
tf021Arr = np.array([(val & 0xFF00)>>8 for val in tf02Arr])
tf022Arr = np.array([(val & 0xFF0000)>>16 for val in tf02Arr])
tf023Arr = np.array([(val & 0xFF000000)>>24 for val in tf02Arr])

tf030Arr = np.array([val & 0xFF for val in tf03Arr])
tf031Arr = np.array([(val & 0xFF00)>>8 for val in tf03Arr])
tf032Arr = np.array([(val & 0xFF0000)>>16 for val in tf03Arr])
tf033Arr = np.array([(val & 0xFF000000)>>24 for val in tf03Arr])


tf100Arr = np.array([val & 0xFF for val in tf10Arr])
tf101Arr = np.array([(val & 0xFF00)>>8 for val in tf10Arr])
tf102Arr = np.array([(val & 0xFF0000)>>16 for val in tf10Arr])
tf103Arr = np.array([(val & 0xFF000000)>>24 for val in tf10Arr])

tf110Arr = np.array([val & 0xFF for val in tf11Arr])
tf111Arr = np.array([(val & 0xFF00)>>8 for val in tf11Arr])
tf112Arr = np.array([(val & 0xFF0000)>>16 for val in tf11Arr])
tf113Arr = np.array([(val & 0xFF000000)>>24 for val in tf11Arr])

tf120Arr = np.array([val & 0xFF for val in tf12Arr])
tf121Arr = np.array([(val & 0xFF00)>>8 for val in tf12Arr])
tf122Arr = np.array([(val & 0xFF0000)>>16 for val in tf12Arr])
tf123Arr = np.array([(val & 0xFF000000)>>24 for val in tf12Arr])

tf130Arr = np.array([val & 0xFF for val in tf13Arr])
tf131Arr = np.array([(val & 0xFF00)>>8 for val in tf13Arr])
tf132Arr = np.array([(val & 0xFF0000)>>16 for val in tf13Arr])
tf133Arr = np.array([(val & 0xFF000000)>>24 for val in tf13Arr])

In [51]:
# Visualize Power Trace:
p = figure(plot_width=1000, plot_height=400)
idx = np.random.randint(0, powerTrace.shape[0])
p.line(range(0, len(powerTrace[idx])), powerTrace[idx])
show(p)
print(tf002Arr)

[174 146 191 145  94 102 232 235  55 218  25 201  28 125  77  56  80 189
 150  57 184 193  23 127 149  20 236 122 198 239  46  67  99  95  50 202
 243 244 100  64  12 129 213  27  39 182 152 173 132 186  92 205 181  43
  30  38 120  15 206 157 226  73 113 247  85  97 231   5  36   0  51 169
 240 223 131 214 114 134 170  76  96  45 133 167 203 168 123   3 156  44
 233  68  87 238  18 194 177  84 166 110 138 159 216  65 195 224  16  47
  11  58 249 117 207 147  35 148  98  74 225 248 187  89 103 172  40  13
 192  22 135 217 104 160  33 165  34  10  86 115  79 144 227 204   9 230
   6 234  17 255 212  83 128 153 111  88 171  62 176  53 124 199 229  54
  93   8 109   7 209 178 196  59 106 252 137  66 163 142  42 251 105 254
  81 210 141 154  19 118  37   2 221 161  72 164 246  24 208  60  29 121
 180 211  71 183  70 245 140 126  63 143 220  21 228 107  26  48 175  49
 253 190 130  69  78 151 219 116  31 222  14  91 242 241  90 112  52 136
 188 119 185 197 139  61  41 108 215  82 155 158  7

### Where do you take the Intermediate Result?

At this point, we have three choices on the location of the intermediate result:

- Output of the SBox opertion, in the GFN4 network
- Output of the T table, in the GFN4 network
- Output of the F0/1 operation in the GFN4 network

The output of each SBox is an 8-bit word, and hence we can perform key search with $2^8$ guesses for each of the 16 Bytes of the Key. On the contrary, the output of the F functions are 32-bits wide and the search space involves $2^{32}$ possibilities for each of the 4 Words of the key. 

However, if the implementation uses a look-up table for the entire F function, without evaluating the intermediate SBox result, the former attack scheme will not work as there won't be any correlation between the intermediate result and the power trace. Hence this is a weaker attack, despite having lower online complexity. 

**Approach:**

- Try the SBox attack first and see if correlations are got with sufficient confidence
- Repeat the attack using T table output as the intermediate result
- Validate the results against the PT-CT pair provided
- If the previous attack fails, try the F0/1 attack

### CPA at the SBox Output

**First Round of CLEFIA:**

PT = P0 | P1 | P2 | P3

T => P0 | P1 ^ WK0 | P2 | P3 ^ WK1

T => P0 ^ RK0 | P1 ^ WK0 | P2 ^ RK1 | P3 ^ WK1

T => S0(P0 ^ RK0) | P1 ^ WK0 | S1(P2 ^ RK1) | P3 ^ WK1  ---> Intermediate Result

P0 ^ RK0 sent to S0 function and P2 ^ RK1 to S1 bytewise, and the HW of the results are correlated against the power trace. To find WK's, the intermediate result used is $P_{2i + 1}$ ^ $WK_i$

**Last Round of CLEFIA:**

**Second Round of CLEFIA:**

In [19]:
def s0(inp):
    return s0Arr[inp]
def s1(inp):
    return s1Arr[inp]

def tf00(inp):
    return tf00Arr[inp]
def tf01(inp):
    return tf01Arr[inp]
def tf02(inp):
    return tf02Arr[inp]
def tf03(inp):
    return tf03Arr[inp]

def tf10(inp):
    return tf10Arr[inp]
def tf11(inp):
    return tf11Arr[inp]
def tf12(inp):
    return tf12Arr[inp]
def tf13(inp):
    return tf13Arr[inp]

def f0(inpBytes, keyBytes):
    return tf00(inpBytes[0] ^ keyBytes[0]) ^ tf01(inpBytes[1] ^ keyBytes[1]) ^ tf02(inpBytes[2] ^ keyBytes[2]) ^ tf03(inpBytes[3] ^ keyBytes[3])
def f1(inpBytes, keyBytes):
    return tf10(inpBytes[0] ^ keyBytes[0]) ^ tf11(inpBytes[1] ^ keyBytes[1]) ^ tf12(inpBytes[2] ^ keyBytes[2]) ^ tf13(inpBytes[3] ^ keyBytes[3])

def intermediate_0(pt_byte, wk_byte):
    return s0(pt_byte ^ wk_byte)
def intermediate_1(pt_byte, wk_byte):
    return s1(pt_byte ^ wk_byte)
def intermediate_xor(pt_byte, wk_byte):
    return pt_byte ^ wk_byte

def intermediate_00(pt_byte, key_byte):
    return tf000Arr[pt_byte ^ key_byte] ^ tf010Arr[pt_byte ^ key_byte] ^ tf020Arr[pt_byte ^ key_byte] ^ tf030Arr[pt_byte ^ key_byte]
def intermediate_01(pt_byte, key_byte):
    return tf001Arr[pt_byte ^ key_byte] ^ tf011Arr[pt_byte ^ key_byte] ^ tf021Arr[pt_byte ^ key_byte] ^ tf031Arr[pt_byte ^ key_byte]
def intermediate_02(pt_byte, key_byte):
    return tf002Arr[pt_byte ^ key_byte] ^ tf012Arr[pt_byte ^ key_byte] ^ tf022Arr[pt_byte ^ key_byte] ^ tf032Arr[pt_byte ^ key_byte]
def intermediate_03(pt_byte, key_byte):
    return tf003Arr[pt_byte ^ key_byte] ^ tf013Arr[pt_byte ^ key_byte] ^ tf023Arr[pt_byte ^ key_byte] ^ tf033Arr[pt_byte ^ key_byte]

def intermediate_10(pt_byte, key_byte):
    return tf10(pt_byte ^ key_byte)
def intermediate_11(pt_byte, key_byte):
    return tf11(pt_byte ^ key_byte)
def intermediate_12(pt_byte, key_byte):
    return tf12(pt_byte ^ key_byte)
def intermediate_13(pt_byte, key_byte):
    return tf13(pt_byte ^ key_byte)

# array holding HW of all 8-bit integers
HW = [ bin(n).count("1") for n in range(0, 256) ]

In [35]:
guessTop5 = []
confScore = []
totTraces, numPoints = powerTrace.shape
alpha = 1.0
activeTraces = int(alpha * totTraces)
print(activeTraces)

attackType = "first"
startPoint = [800, 500, 1100, 15700, 16200, 16000]   # ['RK0', 'WK0/1', 'RK1', 'RK34', 'WK2/3', '']
endPoint = [1000, 700, 1300, 15900, 16800, 16200]

if (attackType == "first"):
    textArr = textInp
    offset = 0
elif (attackType == "last"):
    textArr = cipherText
    offset = 3

HWmatrix = np.zeros((totTraces, 256), dtype= np.uint8)
corrScore = np.zeros((16, 256, numPoints))
printable = []    # for printing the top guessed keys

12000


In [36]:
fmt = "{:02X}<br>{:.3f}"
def format_stat(stat):
    return str(fmt.format(stat[0], stat[1]))

def color_corr_key(row):
    # print(len(row))
    ret = [""] * len(row)
    for i, bnum in enumerate(row):
        if i == 0:
            ret[i] = "color: green"
        else:
            ret[i] = "color: red"
    return ret

In [37]:
# Infer RK0/RK34:
# region to track in the power trace for artefacts from the 1st round

'''
Bytes 0, 1, 2, 3:
StartPoint = 800
EndPoint = 1000 

Bytes 4-7, c-f:
StartPoint = 400
EndPoint = 800 

Bytes 8, 9, a, b:
StartPoint = 1100
EndPoint = 1400?
'''
keyGuess = []
guessTop5 = []
confScore = []

for i in tnrange(0, 4, desc="Attacking RK0/RK34"):
    temp = []
    for keyByte in tnrange(0, 256, desc="Generating Hamming Weights"):
        for traceNum in range(activeTraces):
            if (i % 2 == 0):
                interVal = intermediate_0(textArr[traceNum, i], keyByte)
            else:
                interVal = intermediate_1(textArr[traceNum, i], keyByte)            
            
            HWmatrix[traceNum, keyByte] = bin(interVal).count("1")

        for point in range(startPoint[offset], endPoint[offset]):
            hw = HWmatrix[:activeTraces, keyByte]
            trc = powerTrace[:activeTraces, point]
            corrScore[i, keyByte, point] = np.abs(linregress(hw, trc).slope)
        temp.append((keyByte, np.max(corrScore[i, keyByte])))

    maxScoreVec = corrScore[i].max(axis=1)    # maxScore across tracePoints for each guess
    top5idx = maxScoreVec.argsort()[-5:]
    confScore.append(maxScoreVec[top5idx][-1] / maxScoreVec[top5idx][-2])
    guessTop5.append(("Byte Number: {}".format(i), "Top-5 Indices: {}".format(top5idx), "Top-5 Corr Scores: {}".format(maxScoreVec[top5idx])))
    keyGuess.append(top5idx[-1])
    
    clear_output(wait=True)  # clear the previous output
    display(guessTop5)
    display("Top-5 correlations for Byte {} at points: {}".format(i, corrScore[i].argmax(axis=1)[top5idx]))
    display("Confidence Score: {}".format(confScore), "Key Guesses: {}".format(keyGuess))
        
#    temp.sort(key = lambda x: -x[1])  # sort temp by dom value
#    printable.append(temp)  # add the data in list
#    df = pd.DataFrame(printable).transpose()

#    keyGuess.append(corrScore[i].max(axis=1).argmax())

#    clear_output(wait=True)  # clear the previous output
#    display(df.head().style.format(format_stat).apply(color_corr_key, axis=0))  # display the current status

# if (attackType == "first"):
#     fileName = "../data/keys/RK0"
# else:
#     fileName = "../data/keys/RK34"
# np.save(fileName, keyGuess)

[('Byte Number: 0',
  'Top-5 Indices: [ 96 164 144 211 147]',
  'Top-5 Corr Scores: [0.00156519 0.00172567 0.00191246 0.00221018 0.00593054]'),
 ('Byte Number: 1',
  'Top-5 Indices: [226 109 209   2 125]',
  'Top-5 Corr Scores: [0.0028281  0.00291201 0.00291674 0.00367029 0.01201779]'),
 ('Byte Number: 2',
  'Top-5 Indices: [ 51 187 143 195 184]',
  'Top-5 Corr Scores: [0.00161766 0.00176028 0.00178389 0.00182456 0.00509455]'),
 ('Byte Number: 3',
  'Top-5 Indices: [ 81 228 125  61 242]',
  'Top-5 Corr Scores: [0.00277314 0.002811   0.00290774 0.00321601 0.0052563 ]')]

'Top-5 correlations at points: [831 831 831 831 831]'

'Confidence Score: [2.6832811159025445, 3.2743425399705566, 2.7922086578289522, 1.6344183325076411]'

'Key Guesses: [147, 125, 184, 242]'

In [38]:
# Infer WK0/1 or WK2/3:
# region to track in the power trace for artefacts from the 1st round
'''
Bytes 4-7, c-f:
StartPoint = 400
EndPoint = 800 
'''
keyGuess = []
guessTop5 = []
confScore = []

for i in tnrange(4, 8, desc="Attacking WK0/WK2"):
    for keyByte in tnrange(0, 256, desc="Generating Hamming Weights"):
        for traceNum in range(activeTraces):
            interVal = intermediate_xor(textArr[traceNum, i], keyByte)
            HWmatrix[traceNum, keyByte] = HW[interVal]
       
        hwvec = HWmatrix[:activeTraces, keyByte]
        pTrace = powerTrace[:activeTraces, startPoint[offset+1]:endPoint[offset+1]]
        corrScore[i, keyByte, startPoint[offset+1]:endPoint[offset+1]] = [np.abs(linregress(hwvec, pTrace[:, point]).slope) \
                                                      for point in range(endPoint[offset+1] - startPoint[offset+1])]
        
    maxScoreVec = corrScore[i].max(axis=1)    #maxScore across tracePoints for each guess
    top5idx = maxScoreVec.argsort()[-5:]
    confScore.append(maxScoreVec[top5idx][-1] / maxScoreVec[top5idx][-3])  # top-2 guesses are 1's complements of each other
    guessTop5.append(("Byte Number: {}".format(i), "Top-5 Indices: {}".format(top5idx), "Top-5 Corr Scores: {}".format(maxScoreVec[top5idx])))
    keyGuess.append(top5idx[-1])
    
    clear_output(wait=True)  # clear the previous output
    display(guessTop5)
    display("Top-5 correlations for Byte {} at points: {}".format(i, corrScore[i].argmax(axis=1)[top5idx]))
    display("Confidence Score: {}".format(confScore), "Key Guesses: {}".format(keyGuess))
    
#if (attackType == "first"):
#    fileName = "../data/keys/WK0_probable"
#else:
#    fileName = "../data/keys/WK2_probable"
#np.save(fileName, keyGuess)

[('Byte Number: 0',
  'Top-5 Indices: [ 96 164 144 211 147]',
  'Top-5 Corr Scores: [0.00156519 0.00172567 0.00191246 0.00221018 0.00593054]'),
 ('Byte Number: 1',
  'Top-5 Indices: [226 109 209   2 125]',
  'Top-5 Corr Scores: [0.0028281  0.00291201 0.00291674 0.00367029 0.01201779]'),
 ('Byte Number: 2',
  'Top-5 Indices: [ 51 187 143 195 184]',
  'Top-5 Corr Scores: [0.00161766 0.00176028 0.00178389 0.00182456 0.00509455]'),
 ('Byte Number: 3',
  'Top-5 Indices: [ 81 228 125  61 242]',
  'Top-5 Corr Scores: [0.00277314 0.002811   0.00290774 0.00321601 0.0052563 ]'),
 ('Byte Number: 4',
  'Top-5 Indices: [200  35 220 204  51]',
  'Top-5 Corr Scores: [0.00613454 0.00617584 0.00617584 0.00791717 0.00791717]'),
 ('Byte Number: 5',
  'Top-5 Indices: [101 130 125 138 117]',
  'Top-5 Corr Scores: [0.00664405 0.0066579  0.0066579  0.0086361  0.0086361 ]'),
 ('Byte Number: 6',
  'Top-5 Indices: [ 63  57 198  59 196]',
  'Top-5 Corr Scores: [0.00633597 0.00661258 0.00661258 0.00812255 0.00812

'Top-5 correlations for Byte 7 at points: [572 572 572 572 572]'

'Confidence Score: [2.6832811159025445, 3.2743425399705566, 2.7922086578289522, 1.6344183325076411, 1.2819587150576481, 1.2971201926263454, 1.2283469838359002, 1.2402491640776114]'

'Key Guesses: [51, 117, 196, 107]'

In [41]:
# Infer RK1
# region to track in the power trace for artefacts from the 1st round
'''
Bytes 8, 9, a, b:
StartPoint = 1100
EndPoint = 1400 
'''
keyGuess = []
guessTop5 = []
confScore = []

for i in tnrange(8, 12, desc="Attacking RK1/RK35"):
    for keyByte in tnrange(0, 256, desc="Generating Hamming Weights"):
        for traceNum in range(activeTraces):
            if (i % 2 == 0):
                interVal = intermediate_1(textArr[traceNum, i], keyByte)
            else:
                interVal = intermediate_0(textArr[traceNum, i], keyByte)            
            HWmatrix[traceNum, keyByte] = HW[interVal]
        
        hwvec = HWmatrix[:activeTraces, keyByte]
        pTrace = powerTrace[:activeTraces, startPoint[offset+2]:endPoint[offset+2]]
        corrScore[i, keyByte, startPoint[offset+2]:endPoint[offset+2]] = [np.abs(linregress(hwvec, pTrace[:, point]).slope) \
                                                      for point in range(endPoint[offset+2] - startPoint[offset+2])]
        
    maxScoreVec = corrScore[i].max(axis=1)    #maxScore across tracePoints for each guess
    top5idx = maxScoreVec.argsort()[-5:]
    confScore.append(maxScoreVec[top5idx][-1] / maxScoreVec[top5idx][-2])
    guessTop5.append(("Byte Number: {}".format(i), "Top-5 Indices: {}".format(top5idx), "Top-5 Corr Scores: {}".format(maxScoreVec[top5idx])))
    keyGuess.append(top5idx[-1])
    
    clear_output(wait=True)  # clear the previous output
    display(guessTop5)
    display("Top-5 correlations for Byte {} at points: {}".format(i, corrScore[i].argmax(axis=1)[top5idx]))
    display("Confidence Score: {}".format(confScore), "Key Guesses: {}".format(keyGuess))
   
#if (attackType == "first"):
#    fileName = "../data/keys/RK1"
#else:
#    fileName = "../data/keys/RK35"
#np.save(fileName, keyGuess)

[('Byte Number: 8',
  'Top-5 Indices: [ 95  47 159  63 110]',
  'Top-5 Corr Scores: [0.0030019  0.00327104 0.00336725 0.00348923 0.00427766]'),
 ('Byte Number: 9',
  'Top-5 Indices: [ 87  89 141 120  90]',
  'Top-5 Corr Scores: [0.00262169 0.00281244 0.00287847 0.00351882 0.01206471]'),
 ('Byte Number: 10',
  'Top-5 Indices: [199 214 169 255  37]',
  'Top-5 Corr Scores: [0.00338978 0.00339113 0.00340031 0.00343701 0.00399537]'),
 ('Byte Number: 11',
  'Top-5 Indices: [169  92 161 157 172]',
  'Top-5 Corr Scores: [0.00191771 0.00200447 0.00200801 0.00263395 0.00579721]')]

'Top-5 correlations for Byte 11 at points: [1143 1143 1143 1143 1142]'

'Confidence Score: [1.2259614798089808, 3.428620931017007, 1.1624564586501966, 2.200953823083112]'

'Key Guesses: [110, 90, 37, 172]'

In [42]:
# Infer WK1:
# region to track in the power trace for artefacts from the 1st round
'''
Bytes 4-7, c-f:
StartPoint = 400
EndPoint = 800 
'''
keyGuess = []
guessTop5 = []
confScore = []


for i in tnrange(12, 16, desc="Attacking WK1/WK3"):
    for keyByte in tnrange(0, 256, desc="Generating Hamming Weights"):
        for traceNum in range(activeTraces):
            interVal = intermediate_xor(textArr[traceNum, i], keyByte)
            HWmatrix[traceNum, keyByte] = HW[interVal]
       
        hwvec = HWmatrix[:activeTraces, keyByte]
        pTrace = powerTrace[:activeTraces, startPoint[offset+1]:endPoint[offset+1]]
        corrScore[i, keyByte, startPoint[offset+1]:endPoint[offset+1]] = [np.abs(linregress(hwvec, pTrace[:, point]).slope) \
                                                      for point in range(endPoint[offset+1] - startPoint[offset+1])]
        
    maxScoreVec = corrScore[i].max(axis=1)    #maxScore across tracePoints for each guess
    top5idx = maxScoreVec.argsort()[-5:]
    confScore.append(maxScoreVec[top5idx][-1] / maxScoreVec[top5idx][-3]) # top-2 guesses are 1's complements of each other
    guessTop5.append(("Byte Number: {}".format(i), "Top-5 Indices: {}".format(top5idx), "Top-5 Corr Scores: {}".format(maxScoreVec[top5idx])))
    keyGuess.append(top5idx[-1])
    
    clear_output(wait=True)  # clear the previous output
    display(guessTop5)
    display("Top-5 correlations for Byte {} at points: {}".format(i, corrScore[i].argmax(axis=1)[top5idx]))
    display("Confidence Score: {}".format(confScore), "Key Guesses: {}".format(keyGuess))
    
#if (attackType == "first"):
#    fileName = "../data/keys/WK1_probable"
#else:
#    fileName = "../data/keys/WK3_probable"
#np.save(fileName, keyGuess)

[('Byte Number: 12',
  'Top-5 Indices: [ 74  94 161 177  78]',
  'Top-5 Corr Scores: [0.00612247 0.00615629 0.00615629 0.00786305 0.00786305]'),
 ('Byte Number: 13',
  'Top-5 Indices: [230  21 234 238  17]',
  'Top-5 Corr Scores: [0.00669014 0.00675352 0.00675352 0.0085709  0.0085709 ]'),
 ('Byte Number: 14',
  'Top-5 Indices: [103 158  97 156  99]',
  'Top-5 Corr Scores: [0.00625847 0.00645421 0.00645421 0.00808064 0.00808064]'),
 ('Byte Number: 15',
  'Top-5 Indices: [207  54 201  52 203]',
  'Top-5 Corr Scores: [0.00607846 0.00644057 0.00644057 0.00789676 0.00789676]')]

'Top-5 correlations for Byte 15 at points: [676 676 676 676 676]'

'Confidence Score: [1.2772384648240718, 1.269102344646559, 1.2519956602085116, 1.2260964368757923]'

'Key Guesses: [78, 17, 99, 203]'

### Using T-Tables as Intermediate Result

Split the 8 T-Tables into four, each holding a byte of the result. This approach uses 32 T-Tables in all.

In [18]:
# Infer RK0/RK34:
# region to track in the power trace for artefacts from the 1st round

'''
Bytes 0, 1, 2, 3:
StartPoint = 800
EndPoint = 1000 

Bytes 4-7, c-f:
StartPoint = 400
EndPoint = 800 

Bytes 8, 9, a, b:
StartPoint = 1100
EndPoint = 1400?
'''
keyGuess = []
guessTop5 = []
confScore = []

for i in tnrange(0, 4, desc="Attacking RK0"):
    temp = []
    for keyByte in tnrange(0, 256, desc="Generating Hamming Weights"):
        for traceNum in range(activeTraces):
            if (i == 0):
                interVal = intermediate_00(textInp[traceNum, i], keyByte)
            elif (i == 1):
                interVal = intermediate_01(textInp[traceNum, i], keyByte)
            elif (i == 2):
                interVal = intermediate_02(textInp[traceNum, i], keyByte)
            elif (i == 3):
                interVal = intermediate_03(textInp[traceNum, i], keyByte)
            
            HWmatrix[traceNum, keyByte] = bin(interVal).count("1")

        for point in range(startPoint[offset], endPoint[offset]):
            hw = HWmatrix[:activeTraces, keyByte]
            trc = powerTrace[:activeTraces, point]
            corrScore[i, keyByte, point] = np.abs(linregress(hw, trc).slope)
        temp.append((keyByte, np.max(corrScore[i, keyByte])))

    maxScoreVec = corrScore[i].max(axis=1)    # maxScore across tracePoints for each guess
    top5idx = maxScoreVec.argsort()[-5:]
    confScore.append(maxScoreVec[top5idx][-1] / maxScoreVec[top5idx][-2])
    guessTop5.append(("Byte Number: {}".format(i), "Top-5 Indices: {}".format(top5idx), "Top-5 Corr Scores: {}".format(maxScoreVec[top5idx])))
    keyGuess.append(top5idx[-1])
    
    clear_output(wait=True)  # clear the previous output
    display(guessTop5)
    display("Top-5 correlations for Byte {} at points: {}".format(i, corrScore[i].argmax(axis=1)[top5idx]))
    display("Confidence Score: {}".format(confScore), "Key Guesses: {}".format(keyGuess))
        
#    temp.sort(key = lambda x: -x[1])  # sort temp by dom value
#    printable.append(temp)  # add the data in list
#    df = pd.DataFrame(printable).transpose()

#    keyGuess.append(corrScore[i].max(axis=1).argmax())

#    clear_output(wait=True)  # clear the previous output
#    display(df.head().style.format(format_stat).apply(color_corr_key, axis=0))  # display the current status

# if (attackType == "first"):
#     fileName = "../data/keys/RK0"
# else:
#     fileName = "../data/keys/RK34"
# np.save(fileName, keyGuess)

[(0,
  array([ 98,  44, 110, 186,  11]),
  array([0.00049938, 0.00051278, 0.00053281, 0.00053285, 0.00054694])),
 (1,
  array([204, 203,  53, 199, 235]),
  array([0.00053739, 0.00058844, 0.00059634, 0.00070768, 0.00074085])),
 (2,
  array([104, 149,  45, 100, 181]),
  array([0.00051223, 0.00051264, 0.00051374, 0.00053749, 0.00059108])),
 (3,
  array([104, 233, 151,  63,  87]),
  array([0.00050811, 0.0005087 , 0.0005195 , 0.00056492, 0.00060764]))]

array([1134, 1134, 1143, 1143, 1143])

'Confidence Score: '

[1.0264431768016602,
 1.0468701114117536,
 1.0997014613553246,
 1.0756142196021579]

'Key Guesses: '

[11, 235, 181, 87]

### Second Round Attack

Intermediate Value = output of first round ^ keyGuess

In [43]:
# CPA at T-table output:
def tf00(inp):
    return tf00Arr[inp]

def tf01(inp):
    return tf01Arr[inp]

def tf02(inp):
    return tf02Arr[inp]

def tf03(inp):
    return tf03Arr[inp]

def tf10(inp):
    return tf10Arr[inp]

def tf11(inp):
    return tf11Arr[inp]

def tf12(inp):
    return tf12Arr[inp]

def tf13(inp):
    return tf13Arr[inp]

def f0(inpBytes, keyBytes):
    return tf00(inpBytes[0] ^ keyBytes[0]) ^ tf01(inpBytes[1] ^ keyBytes[1]) ^ tf02(inpBytes[2] ^ keyBytes[2]) ^ tf03(inpBytes[3] ^ keyBytes[3])

def f1(inpBytes, keyBytes):
    return tf10(inpBytes[0] ^ keyBytes[0]) ^ tf11(inpBytes[1] ^ keyBytes[1]) ^ tf12(inpBytes[2] ^ keyBytes[2]) ^ tf13(inpBytes[3] ^ keyBytes[3])

rk0 = np.load("../data/keys/RK0.npy", "r")
rk1 = np.load("../data/keys/RK1.npy", "r")
wk0 = np.load("../data/keys/WK0_probable.npy", "r")
wk1 = np.load("../data/keys/WK1_probable.npy", "r")
wk2 = np.load("../data/keys/WK2_probable.npy", "r")
wk3 = np.load("../data/keys/WK3_probable.npy", "r")

def intermediate_rk2(pt0Bytes, rk0Bytes, pt1Byte, bytePos):
    return (f0(pt0Bytes, rk0Bytes) >> bytePos) & (0xFF) ^ pt1Byte

def intermediate_rk3(pt2Bytes, rk1Bytes, pt3Byte, bytePos):
    return (f1(pt2Bytes, rk1Bytes) >> bytePos) & (0xFF) ^ pt3Byte


In [61]:
keyGuess = []
guessTop5 = []
confScore = []
totTraces, numPoints = powerTrace.shape
alpha = 1.0
activeTraces = int(alpha * totTraces)
print(activeTraces)

# region to track in the power trace for artefacts from the 1st round
startPoint = 1450
endPoint = 1800
'''
Bytes 0, 1, 2, 3:
StartPoint = 800
EndPoint = 1000 

Bytes 4-7, c-f:
StartPoint = 400
EndPoint = 800 

Bytes 8, 9, a, b:
StartPoint = 1100
EndPoint = 1400?
'''
HWmatrix = np.zeros((totTraces, 256), dtype= np.uint8)
corrScore = np.zeros((16, 256, numPoints))

12000


In [62]:
for i in range(0, 4):
    for keyByte in tnrange(0, 256, desc="Generating Hamming Weights"):
        for traceNum in range(activeTraces):

            inp_to_SBox = intermediate_rk2(textInp[traceNum, 0:4], rk0, textInp[traceNum, 4+i], 8 * i)
            #
            if (i % 2 == 0):
                interVal = intermediate_0(inp_to_SBox, keyByte)
            else:
                interVal = intermediate_1(inp_to_SBox, keyByte)
            HWmatrix[traceNum, keyByte] = HW[interVal]
        
        hwvec = HWmatrix[:activeTraces, keyByte]
        pTrace = powerTrace[:activeTraces, startPoint:endPoint]
        corrScore[i, keyByte, startPoint:endPoint] = [np.abs(linregress(hwvec, pTrace[:, point]).slope) \
                                                      for point in range(endPoint - startPoint)]
    
    maxScoreVec = corrScore[i].max(axis=1)    #maxScore across tracePoints for each guess
    top5idx = maxScoreVec.argsort()[-5:]
    confScore.append(maxScoreVec[top5idx][-1] / maxScoreVec[top5idx][-2])
    guessTop5.append(("Byte Number: {}".format(i), "Top-5 Indices: {}".format(top5idx), "Top-5 Corr Scores: {}".format(maxScoreVec[top5idx])))
    keyGuess.append(top5idx[-1])
    
    clear_output(wait=True)  # clear the previous output
    display(guessTop5)
    display("Top-5 correlations for Byte {} at points: {}".format(i, corrScore[i].argmax(axis=1)[top5idx]))
    display("Confidence Score: {}".format(confScore), "Key Guesses: {}".format(keyGuess))
    

[('Byte Number: 0',
  'Top-5 Indices: [110 137   9 161 191]',
  'Top-5 Corr Scores: [0.0004556  0.0004638  0.00046907 0.00047371 0.00050851]'),
 ('Byte Number: 1',
  'Top-5 Indices: [190  35 105 142   4]',
  'Top-5 Corr Scores: [0.00048939 0.00053173 0.00054698 0.00054866 0.00054943]'),
 ('Byte Number: 2',
  'Top-5 Indices: [222  12 236 216  64]',
  'Top-5 Corr Scores: [0.00044894 0.00045771 0.00048234 0.0004856  0.00054386]'),
 ('Byte Number: 3',
  'Top-5 Indices: [106 120 175 219  82]',
  'Top-5 Corr Scores: [0.00043101 0.0004464  0.00045336 0.00054536 0.00060645]')]

'Top-5 correlations for Byte 3 at points: [1714 1705 1714 1705 1705]'

'Confidence Score: [1.073462157435103, 1.0014095366280842, 1.1199769129967154, 1.1120213015202964]'

'Key Guesses: [191, 4, 64, 82]'

In [63]:
for i in range(8, 12):
    for keyByte in tnrange(0, 256, desc="Generating Hamming Weights"):
        for traceNum in range(activeTraces):

            inp_to_SBox = intermediate_rk3(textInp[traceNum, 8:12], rk1, textInp[traceNum, 4+i], 8 * (i - 8))
            #
            if (i % 2 == 0):
                interVal = intermediate_1(inp_to_SBox, keyByte)
            else:
                interVal = intermediate_0(inp_to_SBox, keyByte)
            HWmatrix[traceNum, keyByte] = HW[interVal]
        
        hwvec = HWmatrix[:activeTraces, keyByte]
        pTrace = powerTrace[:activeTraces, startPoint:endPoint]
        corrScore[i, keyByte, startPoint:endPoint] = [np.abs(linregress(hwvec, pTrace[:, point]).slope) \
                                                      for point in range(endPoint - startPoint)]
    
    maxScoreVec = corrScore[i].max(axis=1)    #maxScore across tracePoints for each guess
    top5idx = maxScoreVec.argsort()[-5:]
    confScore.append(maxScoreVec[top5idx][-1] / maxScoreVec[top5idx][-2])
    guessTop5.append(("Byte Number: {}".format(i), "Top-5 Indices: {}".format(top5idx), "Top-5 Corr Scores: {}".format(maxScoreVec[top5idx])))
    keyGuess.append(top5idx[-1])
    
    clear_output(wait=True)  # clear the previous output
    display(guessTop5)
    display("Top-5 correlations for Byte {} at points: {}".format(i, corrScore[i].argmax(axis=1)[top5idx]))
    display("Confidence Score: {}".format(confScore), "Key Guesses: {}".format(keyGuess))
    

[('Byte Number: 0',
  'Top-5 Indices: [110 137   9 161 191]',
  'Top-5 Corr Scores: [0.0004556  0.0004638  0.00046907 0.00047371 0.00050851]'),
 ('Byte Number: 1',
  'Top-5 Indices: [190  35 105 142   4]',
  'Top-5 Corr Scores: [0.00048939 0.00053173 0.00054698 0.00054866 0.00054943]'),
 ('Byte Number: 2',
  'Top-5 Indices: [222  12 236 216  64]',
  'Top-5 Corr Scores: [0.00044894 0.00045771 0.00048234 0.0004856  0.00054386]'),
 ('Byte Number: 3',
  'Top-5 Indices: [106 120 175 219  82]',
  'Top-5 Corr Scores: [0.00043101 0.0004464  0.00045336 0.00054536 0.00060645]'),
 ('Byte Number: 8',
  'Top-5 Indices: [139 241 226  17 251]',
  'Top-5 Corr Scores: [0.00044982 0.0004727  0.00049277 0.00050244 0.00051281]'),
 ('Byte Number: 9',
  'Top-5 Indices: [ 95 182 117 255 129]',
  'Top-5 Corr Scores: [0.00050995 0.00053897 0.00056005 0.0005662  0.00069891]'),
 ('Byte Number: 10',
  'Top-5 Indices: [158 110 153  30 142]',
  'Top-5 Corr Scores: [0.00044741 0.0004823  0.00049969 0.00055467 0.0006

'Top-5 correlations for Byte 11 at points: [1714 1714 1714 1705 1714]'

'Confidence Score: [1.073462157435103, 1.0014095366280842, 1.1199769129967154, 1.1120213015202964, 1.0206439706586024, 1.2343822708557328, 1.1189473974809565, 1.1465473462958837]'

'Key Guesses: [191, 4, 64, 82, 251, 129, 142, 177]'

### Getting all Keys from the Whitening Keys

Since the first and last round attacks give us the whitenining keys, it is possible to generate the entire key set using this information. The catch here however is that, for each byte of the whitening key, there are two potential contenders- key $K$ and its one's complement $\overline{K}$. Hence to find the correct whitening key, we need to perform a search over a space of $2^{16}$ keys and check which combination of WK's result in the round keys RK0, RK1 predicted by CPA. 

The intermediate result used in extracting the Whitening Keys is $XOR(PT, WK)$. Hence, if the HW of the intermediate result for key guess $K$ is $h$, the HW when $\overline{K}$ is used will be $8-h$. This means that if $K$ results in a correlation score of $m$ (= slope of linear fit), $\overline{K}$ will correspond to a slope of $-m$, i.e.- both have the same correlation score.

In [ ]:
# Search for correct whitening keys amongst 2^16 potential candidates
for itn in range(1 << 16):
    
    

### Targeting the T-table outputs

Repeating attack taking the T table output as the intermediate result (for RK0 and RK1). 

**Inferences:**
- In most cases, the Top-1 key guess of the S Box attack matches the Top-1 guess of the T Table Attack
- The S Box attack gives a clearer distinguisher than the T Table attack
- In a couple of cases, the Top-1 key guesses differed. However, the Top-1 key guess of the S Box was present in the Top-5 guesses of the T Table attack. Further, the different between the correlation scores amongst the Top-5 guesses of the T Table was small.

In [ ]:
keyGuess = []
guessTop5 = []
confScore = []
totTraces, numPoints = powerTrace.shape
alpha = 1.0
activeTraces = int(alpha * totTraces)
print(activeTraces)

# region to track in the power trace for artefacts from the 1st round
startPoint = 800
#winSize = 1000
#winStride = 1800
endPoint = 1200
'''
Bytes 0, 1, 2, 3:
StartPoint = 800
EndPoint = 1000 

Bytes 4-7, c-f:
StartPoint = 400
EndPoint = 800 

Bytes 8, 9, a, b:
StartPoint = 1100
EndPoint = 1400?
'''
HWmatrix = np.zeros((totTraces, 256), dtype= np.uint8)
corrScore = np.zeros((16, 256, numPoints))

In [ ]:
#for i in range(0, 16):
for i in [0, 1, 2, 3, 8, 9, 10, 11]:
    for keyByte in tqdm(range(0, 256)):
        for traceNum in range(activeTraces):
            if (i == 0):  
                interVal = tf00(textInp[traceNum, i] ^ keyByte)
            elif (i == 1):
                interVal = tf01(textInp[traceNum, i] ^ keyByte)
            elif (i == 2):
                interVal = tf02(textInp[traceNum, i] ^ keyByte)
            elif (i == 3):
                interVal = tf03(textInp[traceNum, i] ^ keyByte)
            elif (i == 8):
                interVal = tf10(textInp[traceNum, i] ^ keyByte)
            elif (i == 9):
                interVal = tf11(textInp[traceNum, i] ^ keyByte)
            elif (i == 10):
                interVal = tf12(textInp[traceNum, i] ^ keyByte)
            elif (i == 11):
                interVal = tf13(textInp[traceNum, i] ^ keyByte)
            else:
                interVal = intermediate_xor(textInp[traceNum, i], keyByte)
            #HWmatrix[traceNum, keyByte] = HW[interVal]
            HWmatrix[traceNum, keyByte] = bin(interVal).count("1")
       
        #for tracePoint in range(startPoint + i*winStride, startPoint + i*winStride + winSize):
        hwvec = HWmatrix[:activeTraces, keyByte]
        pTrace = powerTrace[:activeTraces, startPoint:endPoint]
        corrScore[i, keyByte, startPoint:endPoint] = [np.abs(linregress(hwvec, pTrace[:, point]).slope) \
                                                      for point in range(endPoint - startPoint)]
        '''
        for tracePoint in range(startPoint, endPoint):
            hwvec = HWmatrix[:activeTraces, keyByte]
            pTrace = powerTrace[:activeTraces, tracePoint]
            corrScore[i, keyByte, tracePoint] = np.abs(linregress(hwvec, pTrace).slope)
        '''
        
    maxScoreVec = corrScore[i].max(axis=1)    #maxScore across tracePoints for each guess
    top5idx = maxScoreVec.argsort()[-5:]
    confScore.append(maxScoreVec[top5idx][-1] / maxScoreVec[top5idx][-2])
    guessTop5.append(("Byte Number: {}".format(i), "Top-5 Indices: {}".format(top5idx), "Top-5 Corr Scores: {}".format(maxScoreVec[top5idx])))
    keyGuess.append(top5idx[-1])
    
    clear_output(wait=True)  # clear the previous output
    display(guessTop5)
    display("Top-5 correlations for Byte {} at points: {}".format(i, corrScore[i].argmax(axis=1)[top5idx]))
    display("Confidence Score: {}".format(confScore), "Key Guesses: {}".format(keyGuess))
    

In [ ]:
'''
    s0Arr =  [0x57, 0x49, 0xd1, 0xc6, 0x2f, 0x33, 0x74, 0xfb,
              0x95, 0x6d, 0x82, 0xea, 0x0e, 0xb0, 0xa8, 0x1c,
              0x28, 0xd0, 0x4b, 0x92, 0x5c, 0xee, 0x85, 0xb1,
              0xc4, 0x0a, 0x76, 0x3d, 0x63, 0xf9, 0x17, 0xaf,
              0xbf, 0xa1, 0x19, 0x65, 0xf7, 0x7a, 0x32, 0x20,
              0x06, 0xce, 0xe4, 0x83, 0x9d, 0x5b, 0x4c, 0xd8,
              0x42, 0x5d, 0x2e, 0xe8, 0xd4, 0x9b, 0x0f, 0x13,
              0x3c, 0x89, 0x67, 0xc0, 0x71, 0xaa, 0xb6, 0xf5,
              0xa4, 0xbe, 0xfd, 0x8c, 0x12, 0x00, 0x97, 0xda,
              0x78, 0xe1, 0xcf, 0x6b, 0x39, 0x43, 0x55, 0x26,
              0x30, 0x98, 0xcc, 0xdd, 0xeb, 0x54, 0xb3, 0x8f,
              0x4e, 0x16, 0xfa, 0x22, 0xa5, 0x77, 0x09, 0x61,
              0xd6, 0x2a, 0x53, 0x37, 0x45, 0xc1, 0x6c, 0xae,
              0xef, 0x70, 0x08, 0x99, 0x8b, 0x1d, 0xf2, 0xb4,
              0xe9, 0xc7, 0x9f, 0x4a, 0x31, 0x25, 0xfe, 0x7c,
              0xd3, 0xa2, 0xbd, 0x56, 0x14, 0x88, 0x60, 0x0b,
              0xcd, 0xe2, 0x34, 0x50, 0x9e, 0xdc, 0x11, 0x05,
              0x2b, 0xb7, 0xa9, 0x48, 0xff, 0x66, 0x8a, 0x73,
              0x03, 0x75, 0x86, 0xf1, 0x6a, 0xa7, 0x40, 0xc2,
              0xb9, 0x2c, 0xdb, 0x1f, 0x58, 0x94, 0x3e, 0xed,
              0xfc, 0x1b, 0xa0, 0x04, 0xb8, 0x8d, 0xe6, 0x59,
              0x62, 0x93, 0x35, 0x7e, 0xca, 0x21, 0xdf, 0x47,
              0x15, 0xf3, 0xba, 0x7f, 0xa6, 0x69, 0xc8, 0x4d,
              0x87, 0x3b, 0x9c, 0x01, 0xe0, 0xde, 0x24, 0x52,
              0x7b, 0x0c, 0x68, 0x1e, 0x80, 0xb2, 0x5a, 0xe7,
              0xad, 0xd5, 0x23, 0xf4, 0x46, 0x3f, 0x91, 0xc9,
              0x6e, 0x84, 0x72, 0xbb, 0x0d, 0x18, 0xd9, 0x96,
              0xf0, 0x5f, 0x41, 0xac, 0x27, 0xc5, 0xe3, 0x3a,
              0x81, 0x6f, 0x07, 0xa3, 0x79, 0xf6, 0x2d, 0x38,
              0x1a, 0x44, 0x5e, 0xb5, 0xd2, 0xec, 0xcb, 0x90,
              0x9a, 0x36, 0xe5, 0x29, 0xc3, 0x4f, 0xab, 0x64,
              0x51, 0xf8, 0x10, 0xd7, 0xbc, 0x02, 0x7d, 0x8e]
'''
'''
    s1Arr =  [0x6c, 0xda, 0xc3, 0xe9, 0x4e, 0x9d, 0x0a, 0x3d,
              0xb8, 0x36, 0xb4, 0x38, 0x13, 0x34, 0x0c, 0xd9,
              0xbf, 0x74, 0x94, 0x8f, 0xb7, 0x9c, 0xe5, 0xdc,
              0x9e, 0x07, 0x49, 0x4f, 0x98, 0x2c, 0xb0, 0x93,
              0x12, 0xeb, 0xcd, 0xb3, 0x92, 0xe7, 0x41, 0x60,
              0xe3, 0x21, 0x27, 0x3b, 0xe6, 0x19, 0xd2, 0x0e,
              0x91, 0x11, 0xc7, 0x3f, 0x2a, 0x8e, 0xa1, 0xbc,
              0x2b, 0xc8, 0xc5, 0x0f, 0x5b, 0xf3, 0x87, 0x8b,
              0xfb, 0xf5, 0xde, 0x20, 0xc6, 0xa7, 0x84, 0xce,
              0xd8, 0x65, 0x51, 0xc9, 0xa4, 0xef, 0x43, 0x53,
              0x25, 0x5d, 0x9b, 0x31, 0xe8, 0x3e, 0x0d, 0xd7,
              0x80, 0xff, 0x69, 0x8a, 0xba, 0x0b, 0x73, 0x5c,
              0x6e, 0x54, 0x15, 0x62, 0xf6, 0x35, 0x30, 0x52,
              0xa3, 0x16, 0xd3, 0x28, 0x32, 0xfa, 0xaa, 0x5e,
              0xcf, 0xea, 0xed, 0x78, 0x33, 0x58, 0x09, 0x7b,
              0x63, 0xc0, 0xc1, 0x46, 0x1e, 0xdf, 0xa9, 0x99,
              0x55, 0x04, 0xc4, 0x86, 0x39, 0x77, 0x82, 0xec,
              0x40, 0x18, 0x90, 0x97, 0x59, 0xdd, 0x83, 0x1f,
              0x9a, 0x37, 0x06, 0x24, 0x64, 0x7c, 0xa5, 0x56,
              0x48, 0x08, 0x85, 0xd0, 0x61, 0x26, 0xca, 0x6f,
              0x7e, 0x6a, 0xb6, 0x71, 0xa0, 0x70, 0x05, 0xd1,
              0x45, 0x8c, 0x23, 0x1c, 0xf0, 0xee, 0x89, 0xad,
              0x7a, 0x4b, 0xc2, 0x2f, 0xdb, 0x5a, 0x4d, 0x76,
              0x67, 0x17, 0x2d, 0xf4, 0xcb, 0xb1, 0x4a, 0xa8,
              0xb5, 0x22, 0x47, 0x3a, 0xd5, 0x10, 0x4c, 0x72,
              0xcc, 0x00, 0xf9, 0xe0, 0xfd, 0xe2, 0xfe, 0xae,
              0xf8, 0x5f, 0xab, 0xf1, 0x1b, 0x42, 0x81, 0xd6,
              0xbe, 0x44, 0x29, 0xa6, 0x57, 0xb9, 0xaf, 0xf2,
              0xd4, 0x75, 0x66, 0xbb, 0x68, 0x9f, 0x50, 0x02,
              0x01, 0x3c, 0x7f, 0x8d, 0x1a, 0x88, 0xbd, 0xac,
              0xf7, 0xe4, 0x79, 0x96, 0xa2, 0xfc, 0x6d, 0xb2,
              0x6b, 0x03, 0xe1, 0x2e, 0x7d, 0x14, 0x95, 0x1d]
'''
    
'''
def f0(inp):
    
    tf00 = np.load(path_to_data+"tf00.npy", "r")
    tf01 = np.load(path_to_data+"tf01.npy", "r")
    tf02 = np.load(path_to_data+"tf02.npy", "r")
    tf03 = np.load(path_to_data+"tf03.npy", "r")
    
    out = tf00[inp[0]] ^ tf01[inp[1]] ^ tf02[inp[2]] ^ tf03[inp[3]]
    return out

def f1(inp):
    
    tf10 = np.load(path_to_data+"tf10.npy", "r")
    tf11 = np.load(path_to_data+"tf11.npy", "r")
    tf12 = np.load(path_to_data+"tf12.npy", "r")
    tf13 = np.load(path_to_data+"tf13.npy", "r")
    
    out = tf10[inp[0]] ^ tf11[inp[1]] ^ tf12[inp[2]] ^ tf13[inp[3]]
    return out

def intermediate(pt, rk0, rk1, wk0, wk1):
    # Assumption: pt is an array of 32-bit integers with 4 elements; rk's and wk's are 32-b ints
        
    # Ensure correct bit-width match!
    out0 = f0(rk0 ^ pt[0])
    out1 = pt[1] ^ wk0
    out2 = f1(rk1 ^ pt[2])
    out3 = pt[3] ^ wk1
    out = [out0, out1, out2, out3]
    return out

'''

'''
#for i in range(0, 16):
#for i in [4, 5, 6, 7, 12, 13, 14, 15]:
for i in [8, 9, 10, 11]:
    for keyByte in tqdm(range(0, 256)):
        for traceNum in range(activeTraces):
            if (i < 4):
                if (i % 2 == 0):
                    interVal = intermediate_0(textInp[traceNum, i], keyByte)
                else:
                    interVal = intermediate_1(textInp[traceNum, i], keyByte)
            elif (i >= 8 and i < 12):
                if (i % 2 == 0):
                    interVal = intermediate_1(textInp[traceNum, i], keyByte)
                else:
                    interVal = intermediate_0(textInp[traceNum, i], keyByte)
            else:
                interVal = intermediate_xor(textInp[traceNum, i], keyByte)
            HWmatrix[traceNum, keyByte] = HW[interVal]
       
        #for tracePoint in range(startPoint + i*winStride, startPoint + i*winStride + winSize):
        hwvec = HWmatrix[:activeTraces, keyByte]
        pTrace = powerTrace[:activeTraces, startPoint:endPoint]
        corrScore[i, keyByte, startPoint:endPoint] = [np.abs(linregress(hwvec, pTrace[:, point]).slope) \
                                                      for point in range(endPoint - startPoint)]
       
    maxScoreVec = corrScore[i].max(axis=1)    #maxScore across tracePoints for each guess
    top5idx = maxScoreVec.argsort()[-5:]
    confScore.append(maxScoreVec[top5idx][-1] / maxScoreVec[top5idx][-2])
    guessTop5.append((i, top5idx, maxScoreVec[top5idx]))
    keyGuess.append(top5idx[-1])
    print(guessTop5)
    print(corrScore[i].argmax(axis=1)[top5idx])
    print("Confidence Score: ", confScore, "Key Guesses: ", keyGuess)
            
# Attack bytes 4-7

# Attack bytes 8-11

# Attack bytes 12-15
'''